In [1]:
%reload_ext autoreload
%autoreload 2

In [75]:
import json

import sqlite3
import instructor
import pandas as pd

from tqdm import tqdm
from openai import OpenAI
from pydantic import ValidationError

from elections import constants
from elections.data_schemas import ArticleSentiment
from elections.prompts.templates import sentiment_template

In [38]:
engine = sqlite3.connect(constants.NEWS_DB)
df = pd.read_sql("SELECT * FROM articles LIMIT 5", engine)
df

,title,description,pubdate,publisher,url,summary,keywords,text,creation_datetime,query
0,Campanha a todo o gás e a prometedora entrevis...,Campanha a todo o gás e a prometedora entrevis...,2023-12-01T08:00:00,SIC Notícias,https://news.google.com/rss/articles/CBMilwFod...,"Para Luís Montenegro, é ""indiferente"" quem suc...","[""costa"", ""sic"", ""luís"", ""prometedora"", ""antón...","Para Luís Montenegro, é ""indiferente"" quem suc...",2024-02-11T17:26:45.678832,'Luis Montenegro'
1,"""Estamos mais preparados do que quaisquer outr...","""Estamos mais preparados do que quaisquer outr...",2023-12-02T08:00:00,Expresso,https://news.google.com/rss/articles/CBMijwFod...,O presidente do PSD defendeu este sábado que o...,"[""governar"", ""quaisquer"", ""programa"", ""preside...",O presidente do PSD defendeu este sábado que o...,2024-02-11T17:26:45.678832,'Luis Montenegro'
2,Luís Montenegro: “Qualquer pessoa tem o direit...,Luís Montenegro: “Qualquer pessoa tem o direit...,2024-02-10T15:30:00,Expresso,https://news.google.com/rss/articles/CBMiwgFod...,O cabeça de lista por Lisboa da Aliança Democr...,"[""sá"", ""tentou"", ""utilizar"", ""sic"", ""socialdem...",O cabeça de lista por Lisboa da Aliança Democr...,2024-02-11T17:40:25.902188,'Luis Montenegro'
3,Luís Montenegro nega “falta de comparência do ...,Luís Montenegro nega “falta de comparência do ...,2024-02-03T08:15:00,SIC Notícias,https://news.google.com/rss/articles/CBMioQFod...,"Para já, Luís Montenegro enfrenta a ameaça do ...","[""política"", ""passos"", ""casa"", ""espinho"", ""luí...",Aos 9 anos ia a pé para a escola e já “tinha a...,2024-02-11T17:40:25.902188,'Luis Montenegro'
4,Luís Montenegro recusa ter medo de debates pol...,Luís Montenegro recusa ter medo de debates pol...,2024-02-08T19:55:48,Público,https://news.google.com/rss/articles/CBMiaGh0d...,Líder social-democrata não responde a repto de...,"[""medo"", ""luís"", ""políticos"", ""ad"", ""debate"", ...",Líder social-democrata não responde a repto de...,2024-02-11T17:40:25.902188,'Luis Montenegro'


In [39]:
print(
    df["text"][0]
)

Para Luís Montenegro, é "indiferente" quem sucederá a António Costa na liderança do Partido Socialista, mas não para Pedro Delgado Alves e José Eduardo Martins, neste Antes Pelo Contrário em podcast. Em entrevista à SIC, o líder do PSD lembrou que tanto José Luís Carneiro como Pedro Nuno Santos "fizeram parte dos governos de António Costa" que, segundo a sua avaliação, tiveram "muito pouco investimento público". Mas é ao antigo ministro das Infraestruturas que dirigiu o maior ataque: "Está associado a três das maiores trapalhadas que este governo trouxe para a política portuguesa". O Antes Pelo Contrário foi emitido a 30 de novembro na SIC Notícias.


In [97]:
# Apply the patch to the OpenAI client
client = instructor.patch(OpenAI())

def get_article_sentiment(article_content):
    system_prompt = sentiment_template.SYSTEM_PROMPT.format(politicians=constants.POLITICIANS)
    user_prompt = sentiment_template.USER_PROMPT.format(article=article_content)
    
    try:
        # to see the raw response: resp[0]._raw_response.model_dump_json(indent=2)
        resp_raw = client.chat.completions.create(
            model=constants.OPENAI_GPT_MODEL,
            response_model=ArticleSentiment,
            max_retries=constants.MAX_RETRIES,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt},
            ],
        )
    except ValidationError:
        return None, None
    resp = [sentiment for sentiment in resp_raw if sentiment.name in article_content]
    return resp_raw, resp

In [98]:
article_sentiments = []
article_sentiments_raw = []

for article_content in tqdm(df["text"]):
    resp_raw, resp = get_article_sentiment(article_content)
    article_sentiments.append(resp)
    article_sentiments_raw.append(resp_raw)

 80%|████████  | 4/5 [09:48<02:27, 147.17s/it]


KeyboardInterrupt: 

In [99]:
article_sentiments_raw 

[None,
 [Sentiment(name='Luís Montenegro', score=0.8, citations=[Citation(quote='este trabalho de preparação do programa e construção de alternativas começou "há quase um ano".', score=0.8, author=None), Citation(quote='"Sem falsas modéstias ou desrespeito para com os nossos adversários, nós estamos mais preparados do que quaisquer outros para poder governar Portugal", considerou.', score=0.8, author=None), Citation(quote='A equipa está atrás de mim, é grande em quantidade e enorme em qualidade"," disse, referindo-se aos 25 coordenadores das áreas temáticas do CEN, uma equipe paritária e na maioria independentes.', score=0.8, author=None), Citation(quote='Aos que pedem "pessoas com mais notoriedade" e "com presenças anteriores em governos", o líder do PSD disse que também as tem na sua equipe "como tem ficado claro e vai continuar a ficar"," mas deixou uma palavra de confiança na "nova geração" que tem estado na base da preparação do programa eleitoral do PSD.', score=0.8, author=None)

In [100]:
article_sentiments

[None,
 [Sentiment(name='Luís Montenegro', score=0.8, citations=[Citation(quote='este trabalho de preparação do programa e construção de alternativas começou "há quase um ano".', score=0.8, author=None), Citation(quote='"Sem falsas modéstias ou desrespeito para com os nossos adversários, nós estamos mais preparados do que quaisquer outros para poder governar Portugal", considerou.', score=0.8, author=None), Citation(quote='A equipa está atrás de mim, é grande em quantidade e enorme em qualidade"," disse, referindo-se aos 25 coordenadores das áreas temáticas do CEN, uma equipe paritária e na maioria independentes.', score=0.8, author=None), Citation(quote='Aos que pedem "pessoas com mais notoriedade" e "com presenças anteriores em governos", o líder do PSD disse que também as tem na sua equipe "como tem ficado claro e vai continuar a ficar"," mas deixou uma palavra de confiança na "nova geração" que tem estado na base da preparação do programa eleitoral do PSD.', score=0.8, author=None)